# Model Load & Display

#### Imports

In [1]:
from autotab import TabCNN
import os
import numpy as np
import pandas as pd

2021-11-29 20:06:20.965161: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-29 20:06:20.965326: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


### Build The model

In [2]:
my_tabcnn = TabCNN.TabCNN()

Using Google cloud buckets
BUCKET_NAME wagon-data-737-sadriwala
DATA_BUCKET_FOLDER data/spec_repr
gs://wagon-data-737-sadriwala/data/spec_repr/
gs://wagon-data-737-sadriwala/saved/
getting ids from gs://wagon-data-737-sadriwala/data/spec_repr/id.csv


In [3]:
model = my_tabcnn.build_model()

2021-11-29 20:06:37.184844: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-11-29 20:06:37.185001: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-11-29 20:06:37.185115: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (LAPTOP-GP5BU3LN): /proc/driver/nvidia/version does not exist
2021-11-29 20:06:37.186618: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 190, 7, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 188, 5, 64)        18496     
                                                                 
 conv2d_2 (Conv2D)           (None, 186, 3, 64)        36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 93, 1, 64)        0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 93, 1, 64)         0         
                                                                 
 flatten (Flatten)           (None, 5952)              0         
                                                        

### Load the weights

In [5]:
os.chdir('../h5-model')
!ls

00_Jazz1-200-B_solo_0.6.csv  full_val0_75acc_weights.h5  happy_birthday.csv
00_Jazz1-200-B_solo.csv      happy_birthday_0.6.csv


In [6]:
model.load_weights('full_val0_75acc_weights.h5')

### Load the npz File

find the smallest file in 0 folder

In [7]:
import numpy as np
data = np.load(  # load the pre -predicted file
    '/mnt/d/data-science/le-wagon/autotab/data/spec_repr/c/00_Jazz1-200-B_solo.npz',
    allow_pickle=True)
file_list = data.files  # files in data
print(file_list)

['repr', 'labels']


### Extract the repr

In [8]:
repr = data['repr']

In [9]:
repr.shape

(621, 192)

### get nearby frames

In [10]:
con_win_size= my_tabcnn.con_win_size
halfwin = con_win_size // 2
full_x = np.pad(
    repr,
    [
        (halfwin, halfwin
         ),  # full x is the entire song padded with halfwin*2 frames
        (0, 0)
    ],
    mode='constant')


In [11]:
x_new = []
for frame_idx in range(0, len(repr)):  # for all frames in the experiment
    sample_x = full_x[frame_idx:frame_idx + my_tabcnn.con_win_size]
    x_new.append(np.expand_dims(np.swapaxes(sample_x, 0, 1), -1))

In [12]:
print(f"shape = ({len(x_new)},{x_new[0].shape}")

shape = (621,(192, 9, 1)


In [13]:
x_new = np.array(x_new, dtype='float32')


### predict using model.predict on the repr

In [14]:
y_pred = model.predict(x_new)

In [15]:
len(y_pred)

621

### make tabs with the prediction

In [16]:
from autotab.TabPrediction import make_squeezed_tab
from autotab.TabPrediction import make_full_tab
#test it out
expanded_tab = make_full_tab(y_pred, len(y_pred))
expanded_tab = expanded_tab-1
expanded_tab.to_csv("00_Jazz1-200-B_solo.csv")

/mnt/d/data-science/le-wagon/autotab/autotab/TabPrediction.py:42: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  tablature[frame_idx] = fret


In [17]:
squeeze_0_6 = make_squeezed_tab(expanded_tab, 3)
squeeze_0_6.to_csv("00_Jazz1-200-B_solo_0.6.csv")
squeeze_0_6

/mnt/d/data-science/le-wagon/autotab/autotab/TabPrediction.py:57: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  squeezed_Tab[batch_idx] = frame_batch.mode(axis='columns')[0]


,0,3,6,9,12,15,18,21,24,27,...,591,594,597,600,603,606,609,612,615,618
e,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,...,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2
B,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,...,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2
G,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,...,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2
D,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,...,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2
A,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,...,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2
E,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,...,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2


In [18]:
import pandas as pd
from autotab.TabDataReprGen import TabDataReprGen

filenames = '../raw_data/experimentmono.wav'
genrep = TabDataReprGen()
x_new = genrep.load_rep_from_raw_file(filenames)
x_new.shape

/mnt/d/data-science/le-wagon/autotab/autotab/TabDataReprGen.py:174: WavFileWarning: Chunk (non-data) not understood, skipping it.
  self.sr_original, data = wavfile.read(file_audio)


(711, 192, 9, 1)

In [19]:
y_pred = model.predict(x_new)

In [20]:
y_pred

array([[[9.92332220e-01, 1.18370447e-03, 4.12837951e-04, ...,
         2.92428394e-05, 2.53412363e-05, 3.78872828e-05],
        [9.86488998e-01, 5.96007681e-04, 8.18358152e-04, ...,
         2.11839451e-05, 1.61393982e-05, 9.02812735e-06],
        [9.70599830e-01, 9.30750393e-04, 3.67674540e-04, ...,
         1.82603653e-05, 5.21702241e-05, 2.97269435e-05],
        [9.57632422e-01, 1.12537644e-03, 1.58200460e-03, ...,
         9.79132892e-05, 4.89120539e-05, 5.35439467e-05],
        [9.55649257e-01, 1.49175792e-03, 1.67476700e-03, ...,
         6.83129983e-05, 3.68000510e-05, 5.21305192e-05],
        [9.81439531e-01, 1.55740324e-03, 6.34908269e-04, ...,
         3.60782215e-05, 2.61031328e-05, 3.37281053e-05]],

       [[9.92291272e-01, 1.18802721e-03, 4.14533715e-04, ...,
         2.95057380e-05, 2.54799397e-05, 3.81403879e-05],
        [9.86226976e-01, 6.06664864e-04, 8.31673213e-04, ...,
         2.16053977e-05, 1.64983794e-05, 9.27234942e-06],
        [9.70293343e-01, 9.51543741e-0

In [21]:
len(y_pred)

711

In [22]:
from autotab.TabPrediction import make_squeezed_tab
from autotab.TabPrediction import make_full_tab
#test it out
expanded_tab = make_full_tab(y_pred, len(y_pred))
expanded_tab = expanded_tab
expanded_tab.to_csv("happy_birthday.csv")

/mnt/d/data-science/le-wagon/autotab/autotab/TabPrediction.py:42: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  tablature[frame_idx] = fret


In [23]:
squeeze_0_6 = make_squeezed_tab(expanded_tab, 3)
squeeze_0_6.to_csv("happy_birthday_0.6.csv")
squeeze_0_6

/mnt/d/data-science/le-wagon/autotab/autotab/TabPrediction.py:57: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  squeezed_Tab[batch_idx] = frame_batch.mode(axis='columns')[0]


,0,3,6,9,12,15,18,21,24,27,...,681,684,687,690,693,696,699,702,705,708
e,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
B,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
G,-1,-1,-1,-1,-1,-1,-1,-1,5,5,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
D,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
A,-1,-1,-1,-1,-1,-1,-1,-1,8,8,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
E,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [24]:
squeeze_1_0 = make_squeezed_tab(expanded_tab, 5)
squeeze_1_0.to_csv("happy_birthday_1.0.csv")
squeeze_1_0

/mnt/d/data-science/le-wagon/autotab/autotab/TabPrediction.py:57: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  squeezed_Tab[batch_idx] = frame_batch.mode(axis='columns')[0]


,0,5,10,15,20,25,30,35,40,45,...,665,670,675,680,685,690,695,700,705,710
e,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
B,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
G,-1,-1,-1,-1,-1,5,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
D,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
A,-1,-1,-1,-1,-1,8,8,8,-1,8,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
E,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [25]:
squeeze_0_8 = make_squeezed_tab(expanded_tab, 4)
squeeze_0_8.to_csv("happy_birthday_0.8.csv")
squeeze_0_8

/mnt/d/data-science/le-wagon/autotab/autotab/TabPrediction.py:57: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  squeezed_Tab[batch_idx] = frame_batch.mode(axis='columns')[0]


,0,4,8,12,16,20,24,28,32,36,...,672,676,680,684,688,692,696,700,704,708
e,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
B,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
G,-1,-1,-1,-1,-1,-1,5,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
D,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
A,-1,-1,-1,-1,-1,-1,8,8,8,8,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
E,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
